In [1]:
import os
import requests
import base64
import json
from jinja2 import Environment, FileSystemLoader

def page_link(a):
    if ' ' in a:
        return a.replace(' ', '_')
    else:
        return a

# Set environment variables
token = os.environ['LAL']
url_base = "https://api.github.com/repos/harshulgupt/json_pvt/contents/jsons/"
headers = {
    "Authorization": "Token " + token
}

# Fetch and decode JSON files
json_files = ["general", "instructor", "post_docs", "grad_students", "undergrads", "alumni"]
data = {}
for json_file in json_files:
    response = requests.get(url_base + json_file + ".json", headers=headers)
    content = response.json()["content"]
    decoded = base64.b64decode(content).decode()
    data[json_file] = json.loads(decoded)

# Create instance of Environment and render template
environment = Environment(loader=FileSystemLoader('templates/'))
environment.globals['page_link'] = page_link
template = environment.get_template('individual_person.html.j2')
for role in [data["instructor"], data["post_docs"], data["grad_students"], data["undergrads"], data["alumni"]]:
    for person in role[1:]:
        filename = f"people/{ page_link(person['name'])}.html"
        content = template.render(general=data['general'], person=person)
        with open(filename, mode='w', encoding='utf-8') as page:
            page.write(content)
